## End to end implementation of Agent based Rag

## Let's install the necessary libraries needed for our project

In [1]:
!pip install langchain langchain_community # popular framework for generative ai
%pip install --upgrade --quiet huggingface_hub
!pip install faiss-cpu # vectorstore
!pip install langchain_huggingface
!pip install chromadb # vectorstore
!pip install langchain_core
!pip install wikipedia # this's our tool on which the agent wiil search the answer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 15.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 66.8 MB/

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_tkn = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN") # for accessing my secret HuggingFace token

from langchain_community.document_loaders import PyPDFLoader # this's for loading our document of interest
from langchain.llms import HuggingFaceHub # for accessing LLM chatmodel from huggingface
from langchain_huggingface import HuggingFaceEndpoint # for accessing huggingface models
from langchain_huggingface import HuggingFaceEmbeddings # embeding the documents in the vectorstore
from langchain_huggingface import ChatHuggingFace # chat model
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter # for splitting the documents into chunks
from langchain_community.vectorstores import FAISS,Chroma # for storing the embeddings of our chunks
from langchain_core.output_parsers import StrOutputParser # parsing output of an llm into desired format
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper  # tools
from langchain.tools.retriever import create_retriever_tool # used to wrap retriever as a tool
from langchain.agents import AgentExecutor, create_react_agent # creating reasoning and action step agent

In [3]:
# loading the documents
loader = PyPDFLoader("/kaggle/input/machinelearning/machine_learning_with_python_tutorial.pdf")
docs = loader.load()

#splitting the documents
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

#creating embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector = FAISS.from_documents(documents, embedding = embeddings)
retriever = vector.as_retriever() # this retriever is used to retrieve answers from the document to the user

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## let's create our chat model, we load it drom hugging face API

In [4]:
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=hf_tkn
)

chat_model = ChatHuggingFace(llm=llm)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

## Tool creation
*WikipediaAPIWrapper - Wrapper around WikipediaAPI.*
*WikipediaQueryRun implements the standard Runnable Interface.*

In [5]:
wrap = WikipediaAPIWrapper(top_k_results = 1)
wikipedia = WikipediaQueryRun(api_wrapper=wrap)

### retriever tool below modify the retriever for it to function as a tool

In [6]:
retriever_tool = create_retriever_tool(retriever,"doc_search","search for contents in a document")

### now this's our overall tool 👇, a combination of both wikipedia tool and retriever tool

In [7]:
tools = [wikipedia,retriever_tool]

## Now let's create our promppt template to our rag system

In [18]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"],
    template="""
You are an intelligent assistant that can use tools to answer questions accurately. You must strictly adhere to the following format for your response:

Question: The question you must answer
Thought: Think step by step about what to do
Action: The action to take, should be one of [{tool_names}]
Action Input: The input to the action
Observation: The result of the action
Thought: I now know the final answer
Final Answer: The final answer to the original input question

IMPORTANT: If you cannot answer, explicitly state so, or take an action using the available tools.

The tools you can use are:
{tools}

Now, begin!

Question: {input}
{agent_scratchpad}
"""
)


### Creation of Reasoning and Action agent (ReAct), it think step by step, first given tools it first goes to the retriever tool
### to obtain a clue about the question, then it goes to the wikipedia tool to obtain another clue
### then finnally it gives an answer using both the clues

In [19]:
agent = create_react_agent(llm, tools, prompt)

In [20]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True) # implements the standard runnable interface

## let's do a sample user input

In [21]:
agent_executor.invoke({"input": "what are the most popular ensemble learning methods?"})



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2245: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Thought: In order to answer this question, we need to understand what ensemble learning is and which methods are most commonly used in practice.

Action: wikipedia
Action Input: "ensemble learning"

ObservationPage: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms alone.
Unlike a statistical ensemble in statistical mechanics, which is usually infinite, a machine learning ensemble consists of only a concrete finite set of alternative models, but typically allows for much more flexible structure to exist among those alternatives.



/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2245: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Thought: Based on the information provided by the Wikipedia page, we can conclude that ensemble learning involves using multiple learning algorithms to improve predictive performance. We now need to determine which methods are most popular.

Action: doc_search
Action Input: "most popular ensemble learning methods"

Observationlearners. 
Ensemble Learning Methods 
The f ollowing are the most popular ensemble learning methods i.e. the methods for 
combining the predictions from different models: 
Bagging  
The term bagging is also known as bootstrap aggregation. In bagging methods, ensemble 
model tries to improve prediction accuracy and d ecrease model variance by combining 
predictions of individual models trained over randomly generated training samples. The 
final prediction of ensemble model will be given by calculating the average of all predictions 
from the individual estimators. One of the best examples of bagging methods are random 
forests. 
Boosting  
In boosting method, the 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2245: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Based on the information provided in the text material, we can conclude that bagging, boosting, and voting are the most popular ensemble learning methods. AdaBoost and Gradient Boosting are two common boosting ensemble algorithms.

Final Answer: Bagging, boosting, and voting are the most popular ensemble learning methods. AdaBoost and Gradient Boosting are two common boosting ensemble algorithms.</s>

> Finished chain.


{'input': 'what are the most popular ensemble learning methods?',
 'output': 'Bagging, boosting, and voting are the most popular ensemble learning methods. AdaBoost and Gradient Boosting are two common boosting ensemble algorithms.</s>'}